<a href="https://colab.research.google.com/github/gkanfer/AI-PS/blob/master/dash.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade plotly
!pip install jupyter-dash -q
!pip install dash-bootstrap-components

In [ ]:
from jupyter_dash import JupyterDash
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc
import pandas as pd
import sys
import numpy as np
np.set_printoptions(threshold=sys.maxsize)

import numpy as np
import pandas as pd
import pymc as pm
import pytensor.tensor as pt
import arviz as az
np.set_printoptions(threshold=sys.maxsize)

import matplotlib.pyplot as plt
from PIL import Image
import seaborn as sns
import os
import re
import glob
import random
import plotnine
from sklearn import preprocessing
from tqdm import tqdm
import plotly.express as px
%config InlineBackend.figure_format = 'retina'
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import plotly.graph_objects as go
import arviz as az
RANDOM_SEED = 8927
rng = np.random.default_rng(RANDOM_SEED)

%config InlineBackend.figure_format = 'retina'
az.style.use("arviz-darkgrid")
import warnings
%config InlineBackend.figure_format = 'retina'
warnings.simplefilter(action="ignore", category=(FutureWarning, UserWarning))
RANDOM_SEED = 8927
np.random.seed(RANDOM_SEED)
az.style.use("arviz-darkgrid")
az.rcParams["stats.hdi_prob"] = 0.89
az.rcParams["stats.ic_scale"] = "deviance"
az.rcParams["stats.information_criterion"] = "waic"
plt.rcParams["figure.figsize"] = (3.5, 2.5) # sinnle column
# #plt.rcParams["figure.figsize"] = (7, 4) # double-column
sns.set_context("paper", font_scale=1.5)
#sns.set_palette("colorblind")
plt.rcParams['figure.dpi'] = 150
plt.rcParams['font.family'] = ['Times New Roman', 'serif']
plt.rcParams['font.family'] = ['serif']
plt.rcParams['font.size'] = 12
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['axes.titlesize'] = 12
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

In [ ]:
from google.colab import files
uploaded = files.upload()
import Simulate_model_colab as Simulate_model

In [ ]:
# genrate model

In [ ]:
app = JupyterDash(__name__)
dropdown_menu = dcc.Dropdown(
  id = "x-value",
  options=[
    {"label": "M", "value": "M"},
    {"label": "H0", "value": "H0"},
    {"label": "Ht", "value": "Ht"},
    {"label": "U", "value": "U"},
  ],
  value="M",
),
dcc.Dropdown(id = "y-value",options=[
    {"label": "M", "value": "M"},
    {"label": "H0", "value": "H0"},
    {"label": "Ht", "value": "Ht"},
    {"label": "U", "value": "U"},
  ],
  value="H0",)


app.layout = dbc.Container([
        html.H1("AIPyS Model simulatoR"),
        html.Hr(),
        dbc.Row([
            dbc.Col([
                dbc.Col([
                    dbc.Label("Number of samples"),
                    dcc.Input(
                                id="samples",
                                type="number",
                                placeholder="Entar number of samples to simulate e.g. 50",
                                value = 50,
                                ),
                    dbc.Label("Number of sgRNA"),
                    dcc.Input(
                                id="sgRNA",
                                type="number",
                                placeholder="Entar number of samples to simulate e.g. 20",
                                value = 20,
                                ),
                    html.Button('Run', id='run-val', n_clicks=0),
                    ], width={"size": 8}),
                dbc.Col([
                    dbc.Label("AIPyS Effect (beta)"),
                    dcc.Dropdown(
                        id = "aipys-eff",
                        options=[
                          {"effect": "yes", "value": "yes"},
                          {"No effect": "no", "value": "no"},
                        ],
                        value="no",
                      )]),
                dbc.Col([
                    dbc.Label("MOI effect (u)"),
                    dcc.Dropdown(
                        id = "moi-eff",
                        options=[
                          {"effect": "yes", "value": "yes"},
                          {"No effect": "no", "value": "no"},
                        ],
                        value="no",
                      )]),
            dbc.Col([
                    dcc.Markdown(r'$Ht \sim (\beta + \tau) \times H0$',dangerously_allow_html=True,mathjax=True),
                    ],width={"size": 4}),
            dbc.Row([
                    html.Div(id='run_content'),
                    dcc.Store(id='samples-out',data=None),
                    dcc.Store(id='sgRNA-out',data=None),
                    html.Div([
                        dcc.Graph(
                                id='violin-plot',
                                style={'width': '50%','display': 'inline-block'}),
                        dropdown_menu,
                        dcc.Graph(
                                id='scatter-plot',
                                style={'width': '50%','display': 'inline-block'})])
                    ])
                ])
      ])])

@app.callback([Output('run_content','children'),Output('samples-out','data'),Output('sgRNA-out','data')],[Input('run-val','n_clicks'),Input('samples', 'value  '),Input('sgRNA', 'value')])
def display_selection(run,samples,sgRNA):
  return [dcc.Dropdown(id='group-drop',options=[{'label': i, 'value': i} for i in range(sgRNA)],value=sgRNA[0])],samp,sg


@app.callback([Output('violin-plot', 'figure'),Output('scatter-plot', 'figure')],[Input('samples-out','data'),Input('sgRNA-out','data'),Input("aipys-eff", "value"),Input("moi-eff", "value"),Input("group-drop", "value"),Input("x-value", "value"),Input("y-value", "value")])
def update_plot(samp,sg,aipys,moi,sel,xval,yval):
    sim = Simulate_model.Simulate_model(sgRNA_num = sg, samples = samp)
    no_effect = [1]*20
    #effect = [1]*2 + [3] + [1]*5
    effect = pm.draw(pm.Gamma.dist(mu = 1, sigma =1), draws=20, random_seed=RANDOM_SEED).tolist()
    #effect_rev = np.linspace(5,1,20).tolist()
    effect_rev = np.repeat(np.arange(5,0,-1),4).tolist()
    if moi == 'yes':
      df,df_tall = sim.sim_base(U = True,z_ef = no_effect, u_ef = effect_rev, sg_ef = effect)
    else:
      df,df_tall = sim.sim_base(U = True,z_ef = no_effect, u_ef = no_effect, sg_ef = effect)

    violin_fig = px.violin(df_tall, x="sgRNA", y="FC", box=True, points="all",
                           hover_data=df.columns, color='Group')

    df_group = df_tall.loc[df_tall['sgRNA'] == sel,:]
    trendline = np.polyfit(df_group[str(xval)], df_group[str(yval)], 1)
    x = df_group[str(xval)].values
    y = df_group[str(yval)].values
    scatter_fig = px.scatter(df_group, x=str(xval), y=str(yval), trendline="ols")
    scatter_fig.add_trace(go.Scatter(x=x, y=trendline[0]*x + trendline[1],
                   mode='lines',
                   name=f'{sel} Trendline',))
    return [violin_fig, scatter_fig]

app.run_server(port=8034,mode='inline')